In [1]:
import yfinance as yf 
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from scipy.stats import norm
from math import sqrt, pi,log, e
import numpy as np


/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def get_delta(price,strike,expiary,vol,call = True,div=0,r=0):
    
    
    try:
        days_to_exp = (dt.datetime.strptime(expiary, "%Y-%m-%d") - dt.datetime.now()).days
        years_to_exp = days_to_exp/365
        r = 0
        div = 0
        sigmaT = vol * (years_to_exp ** 0.5) # problem here??
        d1 =  (log(price / strike) + (r - div + 0.5 * (vol ** 2)) * years_to_exp)/sigmaT
        d2 = d1 - sigmaT
        dfq = e ** (-div * years_to_exp)
    
        if call == True:
    
            delta = [dfq * norm.cdf(d1)] # call
            return(delta[0])
    
        elif call == False:
            delta = [dfq * (norm.cdf(d1) - 1)] #put
            return(delta[0])
    except:
        return(0)


In [3]:
def get_ImpVol():
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-3-e3cf38dfb721>, line 3)

## Calculate skew for stocks with yahoo finance


Calculates the ATM and Delta Skew for stocks

TO DO:

- Calculate implied jump on earnings days.
- implied distribution at a given time
- Volatility surface

In [4]:
tickers = ['GME', 'TSLA', 'RBLX', 'ARKK', 'SPY', 'AAPL', 'USO', 'GLD']
month_delta = 1

In [5]:
skew_df = pd.DataFrame()
skew_df['Ticker'] = tickers
skew_df['Expiry'] = np.nan
skew_df['Stock Price'] = np.nan
skew_df['ATM Skew'] = np.nan 
skew_df['Delta Skew'] = np.nan 
skew_df['Imp. Forward'] = np.nan 


In [6]:
for i in range(len(tickers)):
    try:
        prices = yf.download(tickers[i],start='2020-01-01', progress = False, threads = False)
        stock = yf.Ticker(tickers[i])
        option_dates = stock.options
        date = dt.datetime.now()
        date += relativedelta(months=month_delta) 
        expiry_one_month = min(option_dates, key=lambda x:abs(dt.datetime.strptime(x, "%Y-%m-%d")-date).days)
        current_price = float(prices['Close'].tail(1))

        calls = stock.option_chain(expiry_one_month).calls
        puts = stock.option_chain(expiry_one_month).puts
        #Expirary 
        skew_df.loc[i,'Expiry'] = expiry_one_month
        #Stock Price
        skew_df.loc[i,'Stock Price'] = current_price
        
        
        #ATM Skew
        ATM_put = puts[puts['strike'] == min(puts['strike'], key=lambda x:abs(x-current_price))]
        ATM_call = calls[calls['strike'] == min(calls['strike'], key=lambda x:abs(x-current_price))]
        ATM_skew = float(ATM_put['impliedVolatility']) - float(ATM_call['impliedVolatility']) 
        skew_df.loc[i,"ATM Skew"] = ATM_skew

        #Delta Skew
        calls['delta'] = calls.apply(lambda row: get_delta(price = current_price, 
                                                       strike = row['strike'],
                                                       vol =row['impliedVolatility'],
                                                       expiary = expiry_one_month, call = True), 
                                                       axis=1)
        puts['delta'] = puts.apply(lambda row: get_delta(price = current_price, 
                                                       strike = row['strike'],
                                                       vol =row['impliedVolatility'],
                                                       expiary = expiry_one_month, call = False), 
                                                       axis=1)

        delta_25_put = puts[puts['delta'] == min(puts['delta'], key=lambda x:abs(x--0.25))]
        delta_25_call = calls[calls['delta'] == min(calls['delta'], key=lambda x:abs(x-0.25))]
        delta_50_call = calls[calls['delta'] == min(calls['delta'], key=lambda x:abs(x-0.50))]

        delta_skew = (float(delta_25_put['impliedVolatility']) - float(delta_25_call['impliedVolatility'])) / float(delta_50_call['impliedVolatility'])

        skew_df.loc[i,"Delta Skew"] = delta_skew
        
        #calculate implied forward from put call parity, using 25 delta put strikes
        
        put_strike = float(delta_25_put['strike'])
        put_price = delta_25_put['lastPrice']
        call_price = calls[calls['strike'] == min(calls['strike'], key=lambda x:abs(x-put_strike))]['lastPrice']
        
        forward = float(call_price) - float(put_price) + put_strike
        
        skew_df.loc[i,'Imp. Forward'] = forward
        
    except:
        print("ERROR! INDEX: ", i, ', TICKER: ', tickers[i])


In [7]:
print("Negative values mean higher call implieds")
skew_df

Negative values mean higher call implieds


,Ticker,Expiry,Stock Price,ATM Skew,Delta Skew,Imp. Forward
0,GME,2022-04-29,186.423706,0.364073,-0.001823,176.30
1,TSLA,2022-04-29,1104.159058,0.049514,0.033101,1098.72
2,RBLX,2022-04-29,51.259998,-0.035644,0.027315,50.95
3,ARKK,2022-04-29,71.290001,0.034668,0.200861,70.40
4,SPY,2022-04-29,459.535004,0.010040,0.463419,458.94
5,AAPL,2022-04-29,178.028107,0.022522,0.205765,177.30
6,USO,2022-04-29,75.800003,0.031006,-0.031725,74.65
7,GLD,2022-04-29,178.895004,0.004578,-0.125472,178.19
